# Brain Tumor Detection with RESNET50 Transfer Learning

This Jupyter Notebook demonstrates building a brain tumor detection model using the Resnet50 pre-trained convolutional neural network (CNN) for transfer learning. Transfer learning leverages the knowledge learned by Resnet50 on a massive image dataset (ImageNet) to accelerate your brain tumor detection model's training and potentially improve its performance.

**Steps:**

1. **Import Necessary Libraries**
2. **Load and Preprocess Brain Tumor Dataset**
   - Download and prepare the brain tumor dataset
   - Explore the dataset's content and structure (number of images, classes, etc.)
   - Preprocess images: resizing, normalization, data augmentation (optional)
3. **Prepare Dataloader**
4. **Load the Resnet50 Model**
   - Use the deep learning library FAST.AI to load the Resnet50 model pre-trained on ImageNet.
   - Freeze the convolutional base layers of Resnet50 to retain their learned features.
   - Add custom classification layers suitable for brain tumor detection.
5. **Compile and Train the Model**
  - Define loss function (e.g., binary cross-entropy for binary classification)
   - Choose an optimizer (e.g., Adam)
   - Specify metrics (e.g., accuracy)
   - Train the model on the preprocessed brain tumor dataset
   - Monitor training progress (loss, accuracy) using visualization tools
   - Consider using techniques like early stopping or learning rate scheduling to optimize training
6. **Evaluate the Model**
   - Evaluate the model's performance on a separate test dataset that wasn't used for training.
   - Calculate metrics like accuracy, precision, recall, F1-score, and confusion matrix.
7. **Visualize Results**
   - Visualize intermediate activations using techniques like Grad-CAM to understand which image regions the model focuses on for classification.
   - Generate predictions on new brain tumor images.

**Additional Considerations:**

- Experiment with hyperparameter tuning (e.g., learning rate, batch size) to improve model performance.
- Consider using data augmentation techniques (e.g., random rotations, flips) to artificially increase the dataset size and improve model generalization.


This project can contribute to the advancement of medical image analysis and potentially aid in early diagnosis of brain tumors.


# 1. Import Necessary Libraries

In [ ]:
! pip install --upgrade -q kaggle torchmetrics

In [6]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.nn as nn
from torchvision import models
import torch.optim as optim
from torch.optim import lr_scheduler
from torchmetrics import Accuracy, Precision, Recall, F1Score
import matplotlib.pyplot as plt
%matplotlib inline

import time
import os
import copy

import shutil
import warnings
import random
import numpy as np

In [7]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(42)

warnings.filterwarnings('ignore')

# Load and Preprocess Brain Tumor Images

In [ ]:
from google.colab import files

files.upload()

! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d sartajbhuvaji/brain-tumor-classification-mri

In [ ]:
! unzip /content/brain-tumor-classification-mri.zip

# Create directoties for respective classes

In [8]:
# Define source directories
training_dir = '/content/Testing'
testing_dir = '/content/Training'

# Define target directories
target_images_train_dir = 'Images/train'
target_images_valid_dir = 'Images/valid'
target_labels_train_dir = 'Labels/train'
target_labels_valid_dir = 'Labels/valid'

# Create target directories if they do not exist
os.makedirs(target_images_train_dir, exist_ok=True)
os.makedirs(target_images_valid_dir, exist_ok=True)
os.makedirs(target_labels_train_dir, exist_ok=True)
os.makedirs(target_labels_valid_dir, exist_ok=True)

# Function to copy files
def copy_files(src_dir, target_dir):
    for tumor_type in os.listdir(src_dir):
        tumor_src_dir = os.path.join(src_dir, tumor_type)
        tumor_target_dir = os.path.join(target_dir, tumor_type)
        os.makedirs(tumor_target_dir, exist_ok=True)

        for filename in os.listdir(tumor_src_dir):
            src_file = os.path.join(tumor_src_dir, filename)
            target_file = os.path.join(tumor_target_dir, filename)
            shutil.copyfile(src_file, target_file)

# Copy training images
copy_files(training_dir, target_images_train_dir)
# Copy testing images as validation images
copy_files(testing_dir, target_images_valid_dir)

# Create empty label directories as YOLO requires label files for each image
def create_empty_labels(image_dir, label_dir):
    for tumor_type in os.listdir(image_dir):
        tumor_image_dir = os.path.join(image_dir, tumor_type)
        tumor_label_dir = os.path.join(label_dir, tumor_type)
        os.makedirs(tumor_label_dir, exist_ok=True)

        for filename in os.listdir(tumor_image_dir):
            label_filename = os.path.splitext(filename)[0] + '.txt'
            label_file = os.path.join(tumor_label_dir, label_filename)
            open(label_file, 'a').close()

# Create empty labels for training and validation sets
create_empty_labels(target_images_train_dir, target_labels_train_dir)
create_empty_labels(target_images_valid_dir, target_labels_valid_dir)

print("Dataset reorganization completed successfully.")


# Prepare Dataloader

In [ ]:
# Define transformations for the training and validation data
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Load the datasets with ImageFolder
data_dir = '/content/Images'
image_datasets = {x: datasets.ImageFolder(root=f'{data_dir}/{x}', transform=data_transforms[x]) for x in ['train', 'valid']}

# Create DataLoader for training and validation
dataloaders = {x: DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=4) for x in ['train', 'valid']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'valid']}
class_names = image_datasets['train'].classes


In [ ]:
import json

# Open a file for writing in JSON format
with open('class_names.json', 'w') as outfile:
  # Dump the list of class names to the file
  json.dump(class_names, outfile)

# Model development

In [25]:
# Load the pre-trained resnet model
model = models.resnet50(pretrained=True)


# Freeze all the layers
for param in model.parameters():
    param.requires_grad = False
    
# Unfreeze the last two residual blocks (adjust as needed)
for param in model.layer4[1].parameters():  # Module for block 2 after first conv
    param.requires_grad = True
for param in model.layer4[2:].parameters():  # Modules for block 3 onwards
    param.requires_grad = True

# Modify the classifier to fit the number of classes
num_classes = len(class_names)
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Move the model to GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)



In [ ]:
# Define the loss function and the optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)
# Define the learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

num_classes = len(class_names)

# Define the metrics
accuracy = Accuracy(num_classes = num_classes, task='multiclass').to(device)
precision = Precision(num_classes=num_classes, average='macro', task='multiclass').to(device)
recall = Recall(num_classes=num_classes, average='macro', task='multiclass').to(device)
f1_score = F1Score(num_classes=num_classes, average='macro', task='multiclass').to(device)

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25, save_filename='best_model.pth'):
    since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    patience = 5  # patience for early stopping

    history = {'train_loss': [], 'valid_loss': [], 'train_acc': [], 'valid_acc': [],
               'train_precision': [], 'valid_precision': [], 'train_recall': [], 'valid_recall': [], 'train_f1': [], 'valid_f1': []}

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0
            running_precision = 0.0
            running_recall = 0.0
            running_f1 = 0.0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                running_precision += precision(preds, labels)
                running_recall += recall(preds, labels)
                running_f1 += f1_score(preds, labels)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            epoch_precision = running_precision / len(dataloaders[phase])
            epoch_recall = running_recall / len(dataloaders[phase])
            epoch_f1 = running_f1 / len(dataloaders[phase])

            history[f'{phase}_loss'].append(epoch_loss)
            history[f'{phase}_acc'].append(epoch_acc.item())
            history[f'{phase}_precision'].append(epoch_precision.item())
            history[f'{phase}_recall'].append(epoch_recall.item())
            history[f'{phase}_f1'].append(epoch_f1.item())

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f} Precision: {epoch_precision:.4f} Recall: {epoch_recall:.4f} F1: {epoch_f1:.4f}')

            if phase == 'valid':
                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model_wts = copy.deepcopy(model.state_dict())
                    patience = 5  # Reset patience when new best accuracy is found
                else:
                    patience -= 1

        print()

        if patience == 0:
            print(f'Early stopping triggered after epoch {epoch}.')
            break

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:.4f}')

    # Save the best model weights to a file
    torch.save(best_model_wts, save_filename)

    model.load_state_dict(best_model_wts)
    return model, history


In [ ]:
def evaluate_model(model, dataloaders):
    model.eval()
    running_corrects = 0

    for inputs, labels in dataloaders['valid']:
        inputs = inputs.to(device)
        labels = labels.to(device)

        with torch.no_grad():
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            running_corrects += torch.sum(preds == labels.data)

    acc = running_corrects.double() / dataset_sizes['valid']
    print(f'Validation Accuracy: {acc:.4f}')


In [ ]:
# Train the resnet model
trained_model, training_history = train_model(model, criterion, optimizer, scheduler, num_epochs=25, save_filename = '_best_model.pth')

In [ ]:
# Evaluate the model
evaluate_model(trained_model, dataloaders)

In [ ]:
# Fine tuning 

# Unfreeze all the layers
for param in trained_model.parameters():
    param.requires_grad = False

In [ ]:
# fine tuning
fine_tuned_model, fine_tuning_history = train_model(trained_model, criterion, optimizer, scheduler, num_epochs=25, save_filename = 'fine_tuned_best_model.pth')

In [ ]:
# Evaluate the model
evaluate_model(fine_tuned_model, dataloaders)